**Bayesian upset probability estimation for District Attorney-Public Administrator**

This notebook calculates the "Bayesian upset probability". I.e.,
given a tally of a uniform random sample of ballots, it calculates
the Bayesian posterior probability that the outcome based on the full
set of ballots would be different than a given reported outcome.

The term "Bayesian risk" has been used for the same concept, but it has
other meanings also, and Bayesian approaches
have not (yet?) been shown to be risk-limiting for election auditing.

See [Bayesian Tabulation Audits Explained and Extended](https://arxiv.org/abs/1801.00528)

Written by Ron Rivest 11/3/2018.  (MIT License for code.)

**Voting Rule:** "California top-two", aka first stage of a runoff election. If one candidate wins a majority, then they win.  Otherwise, the top two go on to the general election.


**Caveats** The code has some minor approximations that shouldn't matter,
and that make the code shorter, e.g. using Dirichlet approximation rather
than Dirichlet-multinomial (OK since sample is very small compared to
total number of votes).  It also uses a Haldane prior (setting all pseudocounts to 0)
(OK I believe since sample contains all candidates).  Code could be tweaked
easily for a slightly different Bayesian approach.

In [1]:
from collections import Counter
import numpy as np

## Contest parameters
Configure the parameters for the District Attorney-Public Administrator election contest
in Orange County, CA, 2018 Primary, using the same parameters as the BRAVO calculations
for the same contest in the ocrla-2018p notebook, but it is pretty straightforward
to change the `tally` and `sample_tally` for a different contest.

For more details, see *Report on Orange County 2018 Pilot Risk-Limiting Audits*
by Stephanie Singer and Neal McBurnett, for the Verified Voting Foundation

Four candidates: R, S, M, A (first initial of last name only).
Reported vote counts:

     R: 209,148
     S: 191,346
     M: 121,818
     A: 20,890
     total: 543,202

Tally of sampled ballots:

     R: 22
     S: 23
     M: 9
     A: 6
     total: 60


In [2]:
# Contest: District Attorney-Public Administrator 
tally = Counter({"R":209148, "S":191346, "M":121818, "A":20890})
sample_tally = Counter({"R":22, "S":23, "M":9, "A":6})

# Contest: Sheriff-Coroner
#tally = Counter({"B":265059, "N":166848, "H":104623})
#sample_tally = Counter({"B":28, "N":23, "H":10})

## Helper functions

In [3]:
def contest_outcome(tally):
    """ 
    Return "top two" (aka runoff) outcome for given tally.
    
    tally is Counter mapping candidates to vote counts.
    """

    total_votes = sum(tally.values())
    top_two = tally.most_common(2)
    if top_two[0][1] > total_votes/2:
            return (top_two[0][0],)
    return tuple(sorted([top_two[0][0], top_two[1][0]]))

In [4]:
def gamma(x):
    """ Return usual gamma(x), made 'safe' by returning 0 on 0 input. """
    if x<=0:
        return 0
    else:
        return np.random.gamma(x)

## Simulation

In [5]:
# check outcome for actual election. Reported outcome was ('R', 'S')
print("Election tally:", tally)
print("Total valid votes:", sum(tally.values()))
reported_outcome = contest_outcome(tally)
print("Reported election outcome:", reported_outcome)
print()

print("Sample tally:", sample_tally)
print("Total valid votes:", sum(sample_tally.values()))
print("Election outcome on sample:", contest_outcome(sample_tally))

Election tally: Counter({'R': 209148, 'S': 191346, 'M': 121818, 'A': 20890})
Total valid votes: 543202
Reported election outcome: ('R', 'S')

Sample tally: Counter({'S': 23, 'R': 22, 'M': 9, 'A': 6})
Total valid votes: 60
Election outcome on sample: ('R', 'S')


In [6]:
# Make results reproducable
np.random.seed(42)

In [7]:
# 1,000,000 trials takes about 40 seconds
n_trials = 1000000
counts = Counter()

for trial in range(n_trials):
    trial_tally = Counter({candidate: gamma(votes)
                    for candidate, votes in sample_tally.items()})
    trial_outcome = contest_outcome(trial_tally)
    counts[trial_outcome] += 1

print("Outcome         Percent")
for outcome, cnt in counts.most_common():
    print("  {:13s}  {:6.2%}"
          .format(str(outcome), cnt/n_trials))

Outcome         Percent
  ('R', 'S')     93.48%
  ('S',)          3.40%
  ('R',)          1.81%
  ('M', 'S')      0.73%
  ('M', 'R')      0.48%
  ('A', 'S')      0.07%
  ('A', 'R')      0.04%


## Results

In [8]:
print("Bayesian upset probability estimate: {:.1%}".format(1.0 - (counts[reported_outcome] / n_trials)))

Bayesian upset probability estimate: 6.5%
